In [5]:
# Wumpus World Agent
# Humberto Barrantes
# 2022

# Imports
import numpy as np
from random import randint
import pandas as pd

# Constants
S = 0
B = 1
P = 2
W = 3
V = 4
G = 5

# Agent
class Agent:
    def __init__(self, w_world, start_col, start_row):
        self.w_world = w_world
        self.c = start_col
        self.r = start_row
        self.direction = 'N'
        self.is_alive = True
        self.has_exited = False
        self.kb = np.zeros(
            (w_world.world.shape[0], w_world.world.shape[1], 6),
            dtype=object
        )
        self.score = 0

        for i in range(self.kb.shape[0]):
            for j in range(self.kb.shape[1]):
                for k in range(self.kb.shape[2]):
                    self.kb[i][j][k] = ""

    def print_kb(self):
        for r in range(4):
            for c in range(4):
                for x in range(6):
                    print('{:>2},'.format(self.kb[r][c][x]), end='')
                print('\t', end='')
            print('\n')

    def loc(self):
        return np.array([self.c, self.r])

    # sensors (this must be an array of size 5...)
    def perceives(self):
        pos = self.loc()
        return self.w_world.cell(pos[0], pos[1])

    # returns the list of all adjacent locations (and sense whats there) from current position
    # this can only return immediate locations to current position, does not return diagonal cells
    def adjacent(self):
        rows = self.w_world.world.shape[0]   # 4
        cols = self.w_world.world.shape[1]   # 4
        locations = []
        for row in [self.r - 1, self.r + 1]:
            if 1 <= row <= rows:
                locations.append([(row, self.c), self.w_world.cell(self.c, row)])
        for col in [self.c - 1, self.c + 1]:
            if 1 <= col <= cols:
                locations.append([(self.r, col), self.w_world.cell(col, self.r)])
        return locations

    def move(self, new_r, new_c):
        # allow any in-bounds single-step move
        rows = self.w_world.world.shape[0]
        cols = self.w_world.world.shape[1]
        if 1 <= new_r <= rows and abs(new_r - self.r) + abs(new_c - self.c) == 1:
            self.r = new_r
        if 1 <= new_c <= cols and abs(new_r - self.r) + abs(new_c - self.c) == 1:
            self.c = new_c
        return 0

    def learn_from_pos(self):
        actual_components = self.perceives()

        self.kb[4 - self.r, self.c - 1][S] = ("S" if "S" in actual_components else "~S")
        self.kb[4 - self.r, self.c - 1][B] = ("B" if "B" in actual_components else "~B")
        self.kb[4 - self.r, self.c - 1][P] = ("P" if "P" in actual_components else "~P")
        self.kb[4 - self.r, self.c - 1][W] = ("W" if "W" in actual_components else "~W")
        self.kb[4 - self.r, self.c - 1][V] = ("V")
        self.kb[4 - self.r, self.c - 1][G] = ("G" if "G" in actual_components else "~G")

        for (nrow, ncol), _ in self.adjacent():
            if "S" in actual_components:
                if "~W" not in self.kb[4 - nrow, ncol - 1][W]:
                    self.kb[4 - nrow, ncol - 1][W] = "W?"
            else:
                self.kb[4 - nrow, ncol - 1][W] = "~W"

            if "B" in actual_components:
                if "~P" not in self.kb[4 - nrow, ncol - 1][P]:
                    self.kb[4 - nrow, ncol - 1][P] = "P?"
            else:
                self.kb[4 - nrow, ncol - 1][P] = "~P"

    # this is the main algorithm. The agent must find the best path toward Gold by using
    # propositional logic. This algorithm returns the path taken to Gold or to death.
    def find_gold(self):
        path = []
        gold = False

        while not gold:
            print(f"Agent is on: {self.r}, {self.c}")

            # Step 1: tell everything in actual position
            self.learn_from_pos()
            path.append([self.r, self.c])

            next_xy = []
            self.print_kb()

            if 'G' in self.perceives():
                gold = True
                break

            for (x, y), _ in self.adjacent():
                if "~W" == self.kb[4 - x, y - 1][W]:
                    if "~P" == self.kb[4 - x, y - 1][P]:
                        if "V" != self.kb[4 - x, y - 1][V]:
                            next_xy = [x, y]
                            break

            if len(next_xy) > 0:
                self.move(next_xy[0], next_xy[1])
            else:
                path = path[:-1]
                self.move(path[-1][0], path[-1][1])

            print()

        print(path)
        return "path.... with score:" + str(self.score)

    def _kb_read(self, col, row, kind):
        """kind: 'P' or 'W' -> returns one of {'', 'P', '~P', 'P?', 'W', '~W', 'W?'}"""
        idx = P if kind == 'P' else W
        return self.kb[4 - row, col - 1][idx]

    def entails(self, kind, col, row):
        """
        kind: 'P' for Pit, 'W' for Wumpus
        Returns: ('ENTAILED', True/False) or ('UNKNOWN', None)
        """
        v = self._kb_read(col, row, kind)
        if kind == 'P':
            if v == 'P':
                return ('ENTAILED', True)
            if v == '~P':
                return ('ENTAILED', False)
            return ('UNKNOWN', None)
        else:  # kind == 'W'
            if v == 'W':
                return ('ENTAILED', True)
            if v == '~W':
                return ('ENTAILED', False)
            return ('UNKNOWN', None)

    def pretty_answer(self, text_kind, col, row):
        """text_kind: 'Pit' or 'Wumpus' (for printing)"""
        kind = 'P' if text_kind.lower().startswith('pit') else 'W'
        status, val = self.entails(kind, col, row)
        if status == 'UNKNOWN':
            return f"Query: Is there a {text_kind} at ({col},{row})?  ⇒  UNKNOWN from KB"
        yn = "Yes" if val else "No"
        why = f"(KB contains {'~' if not val else ''}{kind} at ({col},{row}))"
        return f"Query: Is there a {text_kind} at ({col},{row})?  ⇒  {yn}, ENTAILED {why}"


class WumpusWorld:
    def __init__(self, default=True):
        if default:
            self.world = np.matrix([
                ['S', '', 'B', 'P'],
                ['W', 'B,S,G', 'P', 'B'],
                ['S', '', 'B', ''],
                ['', 'B', 'P', 'B']
            ])
        else:
            self.world = self.create_world()

    def create_world(self):
        temp_world = np.zeros((4, 4), dtype=str)
        components = []

        # Add 3 pits
        while len(components) < 3:
            row = randint(0, 3)
            col = randint(0, 3)
            if row != 0 and col != 0 and temp_world[row][col] == '':
                temp_world[row][col] = 'P'
                components.append(['P', [row, col]])

        # Add 1 Wumpus
        while len(components) < 4:
            row = randint(0, 3)
            col = randint(0, 3)
            if row != 0 and col != 0 and temp_world[row][col] == '':
                temp_world[row][col] = 'W'
                components.append(['W', [row, col]])

        # Add 1 Gold
        while len(components) < 5:
            row = randint(0, 3)
            col = randint(0, 3)
            if row != 0 and col != 0 and temp_world[row][col] == '':
                temp_world[row][col] = 'G'
                components.append(['G', [row, col]])

        # Add stench and breeze around Wumpus and Pits
        for t, pos in components:
            if pos[0] + 1 < 4:
                self.create_stench_and_breeze(temp_world, pos[0] + 1, pos[1], t == 'W')
            if pos[0] - 1 >= 0:
                self.create_stench_and_breeze(temp_world, pos[0] - 1, pos[1], t == 'W')
            if pos[1] + 1 < 4:
                self.create_stench_and_breeze(temp_world, pos[0], pos[1] + 1, t == 'W')
            if pos[1] - 1 >= 0:
                self.create_stench_and_breeze(temp_world, pos[0], pos[1] - 1, t == 'W')

        return temp_world

    def create_stench_and_breeze(self, temp_world, row, col, stench):
        if stench:
            if temp_world[row][col] == '':
                temp_world[row][col] = 'S'
            else:
                temp_world[row][col] += ',S'
        else:
            if temp_world[row][col] == '':
                temp_world[row][col] = 'B'
            else:
                temp_world[row][col] += ',B'

    def get_pos(self, wld, col, row):
        return wld[4 - row, col - 1]

    def cell(self, col, row):
        return self.get_pos(self.world, col, row).split(",")

    def view(self):
        return self.world

# Main Program

# --- Truth table and entailment helpers --------------------------------------

def kb_cell_value_3(agent, kind, col, row):
    """
    Map Agent.KB entry -> 'T' | 'F' | '?'
    kind: 'P' or 'W'
    col,row are 1-based.
    """
    idx = P if kind == 'P' else W
    v = agent.kb[4 - row, col - 1][idx]
    if v == 'P' or v == 'W':
        return 'T'
    if v == '~P' or v == '~W':
        return 'F'
    # includes '', 'P?', 'W?' and any unknown placeholder
    return '?'


def kb_truth_table_3(agent):
    """
    Return a 3-valued truth table (T/F/?) for all P(x,y) and W(x,y) in the 4x4 grid.
    Structure: list of dict rows: {'Var': 'P(1,2)', 'Value': 'T'/'F'/'?'}
    """
    rows = []
    for row in range(1, 5):
        for col in range(1, 5):
            rows.append({'Var': f'P({col},{row})', 'Value': kb_cell_value_3(agent, 'P', col, row)})
            rows.append({'Var': f'W({col},{row})', 'Value': kb_cell_value_3(agent, 'W', col, row)})
    return rows


def entails_atomic(agent, kind, col, row, positive=True):
    """
    Check if the KB entails an atomic query:
      positive=True  checks  kind(col,row)      (e.g., P(1,2))
      positive=False checks ~kind(col,row)      (e.g., ~P(1,2))
    Returns: one of {'ENTAILED', 'CONTRADICTED', 'UNKNOWN'}, along with a human string.
    """
    val3 = kb_cell_value_3(agent, kind, col, row)  # 'T'/'F'/'?'
    var_str = f"{kind}({col},{row})"
    if positive:
        if val3 == 'T':  return 'ENTAILED',  f"{var_str} is T in KB ⇒ models(KB) ⊆ models({var_str})"
        if val3 == 'F':  return 'CONTRADICTED', f"{var_str} is F in KB ⇒ models(KB) ⊄ models({var_str}) (KB entails its negation)"
        return 'UNKNOWN', "KB leaves it unknown ⇒ models(KB) not subset of either query or its negation"
    else:
        if val3 == 'F':  return 'ENTAILED',  f"{var_str} is F in KB ⇒ models(KB) ⊆ models(¬{var_str})"
        if val3 == 'T':  return 'CONTRADICTED', f"{var_str} is T in KB ⇒ models(KB) ⊄ models(¬{var_str}) (KB entails the atom)"
        return 'UNKNOWN', "KB leaves it unknown ⇒ models(KB) not subset of either query or its negation"


def print_truth_table_and_check_queries(agent):
    # Build 3-valued truth table
    table = kb_truth_table_3(agent)

    # Pretty print compact grid view for P and W separately
    def grid(kind):
        g = []
        for row in range(1, 5):
            grow = []
            for col in range(1, 5):
                grow.append(kb_cell_value_3(agent, kind, col, row))
            g.append(grow)
        return g

    print("\n=== KB 3-Valued Truth Table (T/F/?) ===")
    print("P(x,y): rows=y (1..4 bottom→top by y), cols=x (1..4 left→right by x)")
    Pg = grid('P')
    Wg = grid('W')
    for r in range(1, 5):
        y = r
        print(f"y={y}  P-row:", Pg[r-1], "   W-row:", Wg[r-1])
    print()

    # Queries (as requested)
    # Query1: Is there a Pit at (1,2)?  => P(1,2)
    q1_res, q1_expl = entails_atomic(agent, kind='P', col=1, row=2, positive=True)
    # Query2: Is there a Wumpus at (2,2)? => W(2,2)
    q2_res, q2_expl = entails_atomic(agent, kind='W', col=2, row=2, positive=True)

    print("=== Entailment as Superset Check (models(KB) ⊆ models(Query)) ===")
    print(f"Q1: P(1,2)?   ⇒ {q1_res}.  {q1_expl}")
    print(f"Q2: W(2,2)?   ⇒ {q2_res}.  {q2_expl}")

    # If you want a long-form list (Var, Value) like a classical "truth table" listing:
    # (This is 3-valued, not all Boolean combinations.)
    # Uncomment below if you prefer a vertical listing.
    for row in table:
      print(f"{row['Var']:>7}: {row['Value']}")


# Now print the 3-valued truth table and check the two queries



# Example (commented out):
# world = WumpusWorld()
# agent = Agent(world, 1, 1)
# print("Agent is on", agent.loc())
# print("Agent Perceives", agent.perceives())

# Initialize default Wumpus World and Agent
wumpus_world = WumpusWorld()
agent = Agent(wumpus_world, 1, 1)

# Display the world matrix
print("Initial Wumpus World:")
print(wumpus_world.world)
print()

# Run the agent to find the gold
result = agent.find_gold()
print(result)

# Let the agent explore using its current policy (fills the KB)
print_truth_table_and_check_queries(agent)
# Now ask the queries using the KB
print(agent.pretty_answer('Pit', 1, 2))     # Query1: Is there a Pit at (1,2)?
print(agent.pretty_answer('Wumpus', 2, 2))  # Query2: Is there a Wumpus at (2,2)?



Initial Wumpus World:
[['S' '' 'B' 'P']
 ['W' 'B,S,G' 'P' 'B']
 ['S' '' 'B' '']
 ['' 'B' 'P' 'B']]

Agent is on: 1, 1
  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

  ,  ,~P,~W,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

~S,~B,~P,~W, V,~G,	  ,  ,~P,~W,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	


Agent is on: 2, 1
  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

  ,  ,~P,W?,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

 S,~B,~P,~W, V,~G,	  ,  ,~P,W?,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

~S,~B,~P,~W, V,~G,	  ,  ,~P,~W,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	


Agent is on: 1, 1
  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

  ,  ,~P,W?,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  ,  ,  ,  ,  ,	

 S,~B,~P,~W, V,~G,	  ,  ,~P,W?,  ,  ,	  ,  ,  ,  ,  ,  ,	  ,  , 